In [2]:
import re
import os
import arrow
import pandas as pd
from tqdm import tqdm_notebook
from IPython.display import display, HTML, FileLink, clear_output

In [3]:
# Path to Bulletin text files
# You'll need to git clone or download the repository at: https://github.com/wragge/trove-texts
# Then change the path below to point to the Bulletin text files
data_dir = '/Volumes/bigdata/mydata/Trove/Bulletin/text'

def get_texts():
    '''
    Generator to loop through filenames.
    '''
    for file in os.listdir(data_dir):
        if file[-4:] == '.txt':
            yield os.path.join(data_dir, file)
            
def get_total_files():
    '''
    Get the total number of files.
    '''
    return len([f for f in os.listdir(data_dir) if f[-4:] == '.txt'])


def get_words(string, direction, window):
    word_list = re.findall(r'\w+', string)
    try:
        if direction == 'before':
            words = word_list[0-window:]
        elif direction == 'after':
            words = word_list[:window]
    except IndexError:
        words = word_list
    return words

def get_contexts(filename, term, window):
    '''
    Although it seems weird to match then tokenise, then get surrounding words,
    this seems much quicker than using re to get specific numbers of words.
    Using TextBlob for tokenisation seems similar, but unnecessary?
    What about just getting all the ngrams and filtering to those with the target in the middle?
    '''
    contexts = []
    with open(filename, 'r') as text_file:
        content = text_file.read().replace('\n', ' ')
        matches = re.finditer(r'\b{}\b'.format(term), content, re.IGNORECASE)
        for match in matches:
            start = match.start()
            end = match.end()
            # Get KWIC string
            kwic = content[start - 50:end + 50]
            # Get lists of words before and after the term
            before = get_words(kwic[:50], 'before', window)
            after = get_words(kwic[-50:], 'after', window)
            contexts.append((kwic, before, after))
    return contexts

def save_as_csv(df, term, window):
    '''
    Save the results as a CSV.
    Convert lists of words into a pipe-separated string.
    '''
    df2 = df.copy()
    df2['before'] = df['before'].str.join(sep='|')
    df2['after'] = df['after'].str.join(sep='|')
    df2 = df2[['id', 'date', 'kwic', 'before', 'after']]
    filename = 'bulletin-{}-words-{}.csv'.format(term, window)
    df2.to_csv(filename, index=False)
    display(FileLink(filename))

def get_all_contexts(term, window=2):
    all_contexts = []
    total = get_total_files()
    for filename in tqdm_notebook(get_texts(), total=total):
        date, id = re.search(r'([0-9\-]+)\-(nla\.obj\-\d+).*\.txt$', filename).groups()
        # clear_output(wait=True)
        # print(filename)
        contexts = get_contexts(filename, term, window)
        for context in contexts:
            all_contexts.append({'id': id, 'date': date, 'kwic': context[0], 'before': context[1], 'after': context[2]})
    df = pd.DataFrame(all_contexts)
    save_as_csv(df, term, window)

In [4]:
get_all_contexts('aliens', window=5)

/Users/tim/mycode/aliens/notebooks/bulletin-aliens-words-5.csv

In [5]:
get_all_contexts('immigrants', window=5)

/Users/tim/mycode/aliens/notebooks/bulletin-immigrants-words-5.csv